# MultiTool Caller Agent

In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [2]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=5, doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [3]:
arxiv.invoke("Multi-Modal Audio Visual Language Models")

'Published: 2025-03-30\nTitle: TouchUp-G: Improving Feature Representation through Graph-Centric Finetuning\nAuthors: Jing Zhu, Xiang Song, Vassilis N. Ioannidis, Danai Koutra, Christos Faloutsos\nSummary: How can we enhance the node features acquired from Pretrained Models (PMs) to\nbetter suit downstream graph learning tasks? Graph Neural Networks (GNNs) have\nbecome the state-of-the-art approach for many high-impact, real-world graph\napplications. For feature-rich graphs, a prevalent practice invol'

In [4]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
Wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
print(Wiki.name)

wikipedia


In [5]:
Wiki.invoke("Rick and Morty")

"Page: Rick and Morty\nSummary: Rick and Morty is an American adult animated science fiction sitcom created by Justin Roiland and Dan Harmon for Cartoon Network's nighttime programming block Adult Swim. The series follows the misadventures of Rick Sanchez, a cynical mad scientist, and his good-hearted but fretful grandson Morty Smith, who split their time between domestic life and interdimensional adventures that take place across an infinite number of realities, often traveling to other planets a"

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults()

/var/folders/x9/7vccwwq12psb87cv4d66wzn00000gn/T/ipykernel_6434/735679136.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily = TavilySearchResults()


In [8]:
tavily.invoke("What is GPT-OSS?")

[{'title': "Run OpenAI's new GPT-OSS (open-source) model on Northflank",
  'url': 'https://northflank.com/blog/self-host-openai-gpt-oss-120b-open-source-chatgpt',
  'content': "Unlike GPT-3.5 or GPT-4, which are closed-source and API-only, GPT-OSS is available to run locally or in your own infrastructure. That means full control over latency, cost, and privacy, especially when paired with a secure, GPU-ready platform like Northflank.\n\nSelf-hosting gpt-oss also means you won’t run into any rate limits. According to OpenAI, gpt-oss 120B’s performance is on par with o4-mini on most benchmarks, which would make it one of the top open-source models, if not the best. [...] Run OpenAI's new GPT-OSS (open-source) model on Northflank\n==========================================================\n\nAI\n\nOpenAI just released GPT-OSS, its first fully open-source large language model family under an Apache 2.0 license. The release includes two models: gpt-oss-20b and gpt-oss-120b, designed for fas

In [9]:
tools=[arxiv, Wiki, tavily]

In [11]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="Gemma2-9b-It")
llm_with_tools=llm.bind_tools(tools)

In [14]:
from pprint import pprint
from langchain_core.messages import HumanMessage, AIMessage

llm_with_tools.invoke([HumanMessage(content=f"What is WACV?"),]).tool_calls

[{'name': 'wikipedia',
  'args': {'query': 'WACV'},
  'id': 'w4cksnpx3',
  'type': 'tool_call'}]

## Chatbot